# Tabular Reinforcement Learning (RL)

Reinforcement Learning (RL) is a learning paradigm where an **agent** interacts with an **environment** by:
- Observing the **state** $s \in S$,
- Taking an **action** $a \in A$,
- Receiving a **reward** $r \in \mathbb{R}$,
- Transitioning to a new **state** $s'$.

The goal is to learn a **policy** $\pi(s)$ that maximizes the long-term reward.

---

## Markov Decision Process (MDP)

An RL problem is typically modeled as an MDP, defined by:
- **States**: $S$
- **Actions**: $A$
- **Reward function**: $R(s,a)$
- **Transition probability**: $P(s' \mid s,a)$
- **Discount factor**: $\gamma \in [0,1]$

---

## Q-Learning (Tabular RL)

Tabular RL stores knowledge in a **Q-table**, where each entry $Q(s,a)$ estimates the value of taking action $a$ in state $s$.

**Update rule (Q-learning, off-policy):**

$$
Q(s,a) \leftarrow Q(s,a) + \alpha\Big[ r + \gamma \max_{a'} Q(s',a') - Q(s,a) \Big]
$$

- $\alpha$: learning rate  
- $\gamma$: discount factor  
- $r$: reward  
- $s'$: next state  

---

## SARSA (On-policy Tabular RL)

**Update rule (SARSA):**

$$
Q(s,a) \leftarrow Q(s,a) + \alpha\Big[ r + \gamma Q(s',a') - Q(s,a) \Big]
$$

SARSA differs from Q-learning because it uses the action $a'$ actually taken by the policy in the next state (on-policy).

---
## Expected SARSA (On-policy Tabular RL)
**Update rule (Expected SARSA):**
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \Big[r + \gamma \cdot \frac{1}{n} \sum_{a'} Q(s', a') - Q(s, a)\Big]
$$
---

*Author: **Samuel Boluwatife Giwa**

In [2]:
import numpy as np


alpha = 0.1        # learning rate
gamma = 0.99       # discount factor
epsilon = 0.3      # exploration probability
episodes = 200    
n_states = 16
n_actions = 4

grid = np.zeros((4, 4))

Q = np.random.rand(n_states, n_actions)

action = {'left': 0 , 'right': 1 ,'up':2,'down':3}

In [ ]:
#environment dynamics
#Grid like environment, index starts from 0

def step(state, action):
    column = state % 4
    row = state // 4
    column_ = column
    row_ = row
    
    
    if action == 0:   #left
        if column_ != 0:
            column_ -= 1
    elif action == 1: #right
        if column_ != 3:
            column_ += 1
    elif action == 2:  #up
        if row_ != 0:
            row_ -= 1
    elif action == 3:   #down
        if row_ != 3:
            row_ += 1

    new_position = ((row_)*4) + (column_)

    return [row_ , column_, new_position]
    

In [4]:
# reward function

from typing import Any, Literal

def reward(position_:int , goal_state: int) -> Literal[10, -1]:
    
    if position_ == goal_state:
        reward = 10
    else:
        reward = -1
    return reward


# Q-Learning Algorithm

In [5]:
episodes = 500
goal_state = 15
epsilon = 0.1

for i in range(episodes):
    current_state = 0
    done = False

    while not done:
        # Epsilon-greedy action selection
        
        
        if np.random.rand() < epsilon:
            current_action = np.random.randint(0, 4)
        else:
            current_action = np.argmax(Q[current_state, :4]) 

        row, col, next_state = step(current_state, current_action)
        r = reward(next_state, goal_state)

        # Q-learning update
        best_next_action = np.argmax(Q[next_state, :])
        Q[current_state, current_action] += alpha * (
            r + gamma * Q[next_state, best_next_action] - Q[current_state, current_action]
        )

        current_state = next_state

        if current_state == goal_state:
            done = True


# SARSA Algorithm

In [6]:
# SARSA


episodes = 500
goal_state = 15
epsilon = 0.1



for i in range(episodes):
    current_state = 0
    done = False

    while not done:
        # Epsilon-greedy action selection
        
        
        if np.random.rand() < epsilon:
            current_action = np.random.randint(0, 4)
        else:
            current_action = np.argmax(Q[current_state, :4]) 

        row, col, next_state = step(current_state, current_action)
        r = reward(next_state, goal_state)


        if np.random.rand() < epsilon:
            next_action = np.random.randint(0, 4)
        else:
            next_action = np.argmax(Q[next_state, :])
        
        
        # SARSA update
        best_next_action = np.argmax(Q[next_state, :])
        Q[current_state, current_action] += alpha * (
            r + gamma * Q[next_state, next_action] - Q[current_state, current_action]
        )

        current_state = next_state

        if current_state == goal_state:
            done = True



## EXPECTED SARSA

In [11]:
# EXPECTED SARSA

episodes = 500
goal_state = 15
epsilon = 0.1
n=4

for i in range(episodes):
    current_state = 0
    done = False

    while not done:
        # Epsilon-greedy action selection
        
        
        if np.random.rand() < epsilon:
            current_action = np.random.randint(0, 4)
        else:
            current_action = np.argmax(Q[current_state, :4]) 

        row, col, next_state = step(current_state, current_action)
        r = reward(next_state, goal_state)
        
        
        #summation of Q values of next state over all posible actions
        
        sum_Q = np.sum(Q[next_state,:n])
        
        

        # Q-learning update
        best_next_action = np.argmax(Q[next_state, :])
        Q[current_state, current_action] += alpha * (
            r + gamma * ((1-epsilon)*Q[next_state, best_next_action] + epsilon/n * sum_Q )- Q[current_state, current_action]
        )

        current_state = next_state

        if current_state == goal_state:
            done = True


## DOUBLE Q-LEARNING

In [12]:
# double Q learning


episodes = 500
goal_state = 15
epsilon = 0.1
Q1 = np.random.rand(n_states, n_actions)
Q2 = np.random.rand(n_states, n_actions)

for i in range(episodes):
    current_state = 0
    done = False

    while not done:
        # Epsilon-greedy action selection
        
        
        if np.random.rand() < epsilon:
            current_action = np.random.randint(0, 4)
        else:
            current_action = np.argmax(Q1[current_state, :] + Q2[current_state, :])
            
            
        row, col, next_state = step(current_state, current_action)
        r = reward(next_state, goal_state)
        
        if np.random.rand() < 0.5:
            best_next_action = np.argmax(Q1[next_state, :])
            Q1[current_state, current_action] += alpha * (
            r + gamma * Q2[next_state, best_next_action] - Q1[current_state, current_action]
        )
        else:
           best_next_action = np.argmax(Q2[next_state, :])
           Q2[current_state, current_action] += alpha * (
            r + gamma * Q1[next_state, best_next_action] - Q2[current_state, current_action]
        )

        current_state = next_state

        if current_state == goal_state:
            done = True
